In [ ]:
!apt install swig cmake ffmpeg xvfb python3-opengl
!pip install stable-baselines3==2.0.0a5 gymnasium[box2d] huggingface_sb3 pyvirtualdisplay imageio[ffmpeg]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libfontenc1 libglu1-mesa libxfont2 libxkbfile1 swig4.0 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  libgle3 python3-numpy swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  freeglut3 libfontenc1 libglu1-mesa libxfont2 libxkbfile1 python3-opengl swig swig4.0
  x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 14 newly installed, 0 to remove and 45 not upgraded.
Need to get 9,755 kB of archives.
After this operation, 25.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 freeglut3 amd64 2.8.1-6 [74.0 kB]
Get:2 http://

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
import gymnasium as gym
env = gym.make("BipedalWalker-v3", hardcore=True)
env.reset()

(array([ 2.7474400e-03,  8.4062879e-07, -1.0962646e-04, -1.6000004e-02,
         9.2177480e-02,  2.5450226e-04,  8.6013079e-01,  1.2009910e-03,
         1.0000000e+00,  3.2576125e-02,  2.5447927e-04,  8.5369474e-01,
        -1.7315547e-04,  1.0000000e+00,  4.4081384e-01,  4.4581994e-01,
         4.6142259e-01,  4.8954999e-01,  5.3410256e-01,  6.0246080e-01,
         7.0914859e-01,  8.8593149e-01,  1.0000000e+00,  1.0000000e+00],
       dtype=float32),
 {})

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())

_____OBSERVATION SPACE_____ 

Observation Space Shape (24,)
Sample observation [-0.8929907   0.43108338  2.82856     1.5274969   1.8848877   2.9114766
  2.8843513   3.202873    0.48891458  1.5213745  -0.13579296  3.0431268
  2.5980642   1.6175705   0.27005756 -0.15475436  0.17067702  0.8261023
  0.6621966  -0.78926146  0.1352147   0.60816765  0.25307906 -0.12157088]


In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.shape)
print("Action Space Sample", env.action_space.sample())


 _____ACTION SPACE_____ 

Action Space Shape (4,)
Action Space Sample [0.1109468  0.36154255 0.3616253  0.7226063 ]


In [ ]:
from stable_baselines3.common.env_util import make_vec_env
env = make_vec_env('BipedalWalker-v3', n_envs=1)

In [ ]:
from stable_baselines3 import TD3  # Import TD3 algorithm

# Use TD3 algorithm
model = TD3(
    policy='MlpPolicy',
    env=env,
    verbose=1
)

Using cuda device


In [ ]:
from wasabi import Printer
import numpy as np
from stable_baselines3.common.base_class import BaseAlgorithm
from pathlib import Path
import tempfile
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecEnv,
    VecVideoRecorder,
)

In [ ]:
msg = Printer()

In [ ]:
def generate_replay(
    model: BaseAlgorithm,
    eval_env: VecEnv,
    video_length: int,
    is_deterministic: bool,
    local_path: Path,
):
    """
    Generate a replay video of the agent
    :param model: trained model
    :param eval_env: environment used to evaluate the agent
    :param video_length: length of the video (in timesteps)
    :param is_deterministic: use deterministic or stochastic actions
    :param local_path: path of the local repository
    """
    # This is another temporary directory for video outputs
    # SB3 created a -step-0-to-... meta files as well as other
    # artifacts which we don't want in the repo.
    with tempfile.TemporaryDirectory() as tmpdirname:
        # Step 1: Create the VecVideoRecorder
        env = VecVideoRecorder(
            eval_env,
            tmpdirname,
            record_video_trigger=lambda x: x == 0,
            video_length=video_length,
            name_prefix="",
        )

        obs = env.reset()
        lstm_states = None
        episode_starts = np.ones((env.num_envs,), dtype=bool)

        try:
            for _ in range(video_length):
                action, lstm_states = model.predict(
                    obs,
                    state=lstm_states,
                    episode_start=episode_starts,
                    deterministic=is_deterministic,
                )
                obs, _, episode_starts, _ = env.step(action)

            # Save the video
            env.close()

            # Convert the video with x264 codec
            inp = env.video_recorder.path
            out = local_path
            os.system(f"ffmpeg -y -i {inp} -vcodec h264 {out}".format(inp, out))
            print(f"Video saved to: {out}"  )
        except KeyboardInterrupt:
            pass
        except Exception as e:
            msg.fail(str(e))
            # Add a message for video
            msg.fail(
                "We are unable to generate a replay of your agent"
            )

In [19]:
model.learn(total_timesteps=580000)

# Save the model
model_name = "vpg-BipedalWalker-v3"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 434      |
|    ep_rew_mean     | -98.5    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 151      |
|    time_elapsed    | 11       |
|    total_timesteps | 1736     |
| train/             |          |
|    actor_loss      | -0.562   |
|    critic_loss     | 19.8     |
|    learning_rate   | 0.001    |
|    n_updates       | 1691     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | -113     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 151      |
|    time_elapsed    | 12       |
|    total_timesteps | 1916     |
| train/             |          |
|    actor_loss      | 0.382    |
|    critic_loss     | 41.8     |
|    learning_rate   | 0.001    |
|    n_updates       | 1871     |
--------------

In [20]:
import os
video_dir = "."
video_name = "replay.mp4"
env_id = "BipedalWalker-v3"

In [21]:
generate_replay(
    model = model,
    eval_env=DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))]),
    video_length=1000,
    is_deterministic=True,
    local_path=os.path.join(video_dir, video_name)
)

Saving video to /tmp/tmptm4fy90_/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmptm4fy90_/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmptm4fy90_/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmptm4fy90_/-step-0-to-step-1000.mp4
Video saved to: ./replay.mp4


In [22]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
from stable_baselines3.common.evaluation import evaluate_policy

eval_env = Monitor(gym.make("BipedalWalker-v3"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=225.24 +/- 106.62913584392817
